In [1]:
import pandas as pd
import time
import requests
from bs4 import BeautifulSoup
import json
import random
import time
from datetime import datetime
import datetime
import os
import numpy as np 
from tqdm import tqdm
from configparser import ConfigParser

# Youtube Channels stats

### Requests Option 1

In [14]:
config = ConfigParser()
config.read('config.cfg')

API_KEY = config['secrets']['api_key']
CHANNEL_ID = "UC_x5XG1OV2P6uZZ5FSM9Ttw"

url_channel_stats = f'https://youtube.googleapis.com/youtube/v3/channels?part=statistics&id={CHANNEL_ID}&key={API_KEY}'
response_channels = requests.get(url_channel_stats)
channel_stats = json.loads(response_channels.content)

In [16]:
channel_stats

{'kind': 'youtube#channelListResponse',
 'etag': 'mQT4CtRMgXvQHoHnDlq-GcQevR4',
 'pageInfo': {'totalResults': 1, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#channel',
   'etag': '970AOUV5s6o0rVo2fJF6UrvsGF4',
   'id': 'UC_x5XG1OV2P6uZZ5FSM9Ttw',
   'statistics': {'viewCount': '237195054',
    'subscriberCount': '2330000',
    'hiddenSubscriberCount': False,
    'videoCount': '5962'}}]}

### Requests Option 2 using googleapliclient

In [20]:
from googleapiclient.discovery import build

In [21]:
config = ConfigParser()
config.read('config.cfg')

API_KEY = config['secrets']['api_key']
CHANNEL_ID = "UC_x5XG1OV2P6uZZ5FSM9Ttw"

youtube = build("youtube", "v3", developerKey=API_KEY)

In [22]:
request = youtube.channels().list(
    part="statistics",
    id=CHANNEL_ID
)

In [23]:
response = request.execute()

In [24]:
response

{'kind': 'youtube#channelListResponse',
 'etag': 'mQT4CtRMgXvQHoHnDlq-GcQevR4',
 'pageInfo': {'totalResults': 1, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#channel',
   'etag': '970AOUV5s6o0rVo2fJF6UrvsGF4',
   'id': 'UC_x5XG1OV2P6uZZ5FSM9Ttw',
   'statistics': {'viewCount': '237195054',
    'subscriberCount': '2330000',
    'hiddenSubscriberCount': False,
    'videoCount': '5962'}}]}

In [9]:
channel_stats = response["items"][0]["statistics"]

date = pd.to_datetime("today").strftime("%y-%m-%d")

data_channel = {
    "Created_at": date,
    "Total_Views": int(float(channel_stats["viewCount"])),
    "Subscribers": int(float(channel_stats["subscriberCount"])),
    "Video_count": int(float(channel_stats["videoCount"]))
}

data_channel

{'Created_at': '23-12-27',
 'Total_Views': 237179646,
 'Subscribers': 2330000,
 'Video_count': 5959}

In [10]:
data_channel["Subscribers"]

2330000

# Data function

In [11]:
def get_stats(api_key, channel_id):
    youtube = build("youtube", "v3", developerKey=api_key)
    request = youtube.channels().list(
        part="statistics",
        id=channel_id
    )
    response = request.execute()
    
    channel_stats = response["items"][0]["statistics"]
    date = pd.to_datetime("today").strftime("%Y-%m-%d")

    data_channel = {
        "Created_at": date,
        "Total_Views": int(float(channel_stats["viewCount"])),
        "Subscribers": int(float(channel_stats["subscriberCount"])),
        "Video_count": int(float(channel_stats["videoCount"]))
    }

    return data_channel

In [12]:
get_stats(API_KEY, CHANNEL_ID)

{'Created_at': '2023-12-27',
 'Total_Views': 237179646,
 'Subscribers': 2330000,
 'Video_count': 5959}

# Channels to analize

In [13]:
channels_name = ["Codecademy", "GitHub", "Amazon Web Services", "Microsoft Azure"]
channels_ids = ["UC5CMtpogD_P3mOoeiDHD5eQ", "UC7c3Kb6jYCRj4JOHHZTxKsQ", "UCd6MoB9NC6uYN2grvUNT-Zg", "UC0m-80FnNY2Qb7obvTL_2fA"]

In [14]:
channels = {
    "Channel_name": channels_name,
    "Channel_id": channels_ids
}

df_channels = pd.DataFrame(channels)

df_channels

,Channel_name,Channel_id
0,Codecademy,UC5CMtpogD_P3mOoeiDHD5eQ
1,GitHub,UC7c3Kb6jYCRj4JOHHZTxKsQ
2,Amazon Web Services,UCd6MoB9NC6uYN2grvUNT-Zg
3,Microsoft Azure,UC0m-80FnNY2Qb7obvTL_2fA


In [15]:
get_stats(API_KEY, df_channels["Channel_id"][1])

{'Created_at': '2023-12-27',
 'Total_Views': 14681711,
 'Subscribers': 340000,
 'Video_count': 1705}

In [16]:
def channel_stats(df, api_key):
    
    date = []
    views = []
    suscriber= []
    video_count = []
    channel_name = []
    
    
    for i in tqdm(range(len(df)), colour = "green"):
        
        stats_temp = get_stats(API_KEY, df_channels["Channel_id"][i])
        
        date.append(stats_temp["Created_at"])
        views.append(stats_temp["Total_Views"])
        suscriber.append(stats_temp["Subscribers"])
        video_count.append(stats_temp["Video_count"])
        channel_name.append(df["Channel_name"][i])

        
    data = {
        "Channel_name": channel_name,
        "Subscribers": suscriber,
        "Video_Count": video_count,
        "Total_Views": views,
        "created_at": date
    }
    
    df_channels_final = pd.DataFrame(data)
    
    return df_channels_final
    

In [32]:
df_channels

,Channel_name,Channel_id
0,Codecademy,UC5CMtpogD_P3mOoeiDHD5eQ
1,GitHub,UC7c3Kb6jYCRj4JOHHZTxKsQ
2,Amazon Web Services,UCd6MoB9NC6uYN2grvUNT-Zg
3,Microsoft Azure,UC0m-80FnNY2Qb7obvTL_2fA


In [33]:
channels_df = channel_stats(df_channels, API_KEY)

100%|██████████| 4/4 [00:01<00:00,  2.82it/s]


In [37]:
channels_df

,Channel_name,Subscribers,Video_Count,Total_Views,created_at
0,Codecademy,289000,270,28801439,2023-12-27
1,GitHub,340000,1705,14681711,2023-12-27
2,Amazon Web Services,722000,15932,349611877,2023-12-27
3,Microsoft Azure,303000,1506,24624056,2023-12-27


In [34]:
channels_df.to_csv("channels_to_analize.csv", encoding='utf-8', index=False)

In [35]:
test = pd.read_csv("channels_to_analize.csv")

In [38]:
test

,Channel_name,Subscribers,Video_Count,Total_Views,created_at
0,Codecademy,289000,270,28801439,2023-12-27
1,GitHub,340000,1705,14681711,2023-12-27
2,Amazon Web Services,722000,15932,349611877,2023-12-27
3,Microsoft Azure,303000,1506,24624056,2023-12-27
